In [15]:
import findspark

In [16]:
findspark.init("C:\\spark")

In [17]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [18]:
spark = SparkSession.builder \
    .master("local") \
    .appName("pyspark_giris") \
    .getOrCreate()

sc = spark.sparkContext

In [19]:
sc

<SparkContext master=local appName=pyspark_giris>

In [20]:
sc.version

'3.5.2'

In [21]:
sc.appName

'pyspark_giris'

In [22]:
#sc.stop() pc'yi kapatmadan uygulanmalı